In [1]:
# !wget https://zenodo.org/records/15038364/files/TaskB.zip

# # Unzip
# !unzip TaskB.zip -d taskB

# !wget https://ec.europa.eu/esco/download/ESCO%20dataset%20-%20v1.2.0%20-%20classification%20-%20en%20-%20csv.zip

# !unzip 'ESCO dataset - v1.2.0 - classification - en - csv.zip' -d data

In [ ]:
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
from sklearn.metrics import average_precision_score
from tqdm import tqdm
import random
import pandas as pd
import numpy as np
import json
import os
import ast
from sentence_transformers import SentenceTransformer, losses, SentenceTransformerTrainer
from sentence_transformers.util import batch_to_device, cos_sim
from datasets import Dataset
import subprocess
import difflib
from functools import partial
# from sklearn.model_selection import train_test_split
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# !git clone https://github.com/TalentCLEF/talentclef25_evaluation_script.git > /dev/null
!pip install -r talentclef25_evaluation_script/requirements.txt > /dev/null
encoder_type = 'esco'

# # Set your HF token & username as environment variables
# os.environ["HF_TOKEN"] = user_secrets.get_secret("hf")

2025-05-27 12:58:57.996834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748350738.224217      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748350738.290087      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df_esco_skills = pd.read_csv('data/skills_en.csv')
df_esco_skills

,conceptType,conceptUri,skillType,reuseLevel,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,definition,inScheme,description
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0005c151-5b5a...,skill/competence,sector-specific,manage musical staff,manage staff of music\ncoordinate duties of mu...,NaN,released,2023-11-30T15:53:37.136Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Assign and manage staff tasks in areas such as...
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00064735-8fad...,skill/competence,occupation-specific,supervise correctional procedures,oversee prison procedures\nmanage correctional...,NaN,released,2023-11-30T15:04:00.689Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Supervise the operations of a correctional fac...
2,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/000709ed-2be5...,skill/competence,sector-specific,apply anti-oppressive practices,apply non-oppressive practices\napply an anti-...,NaN,released,2023-11-28T10:45:53.54Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Identify oppression in societies, economies, c..."
3,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0007bdc2-dd15...,skill/competence,sector-specific,control compliance of railway vehicles regulat...,monitoring of compliance with railway vehicles...,NaN,released,2023-11-30T16:29:18.273Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Inspect rolling stock, components and systems ..."
4,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00090cc1-1f27...,skill/competence,cross-sector,identify available services,establish available services\ndetermine rehabi...,NaN,released,2023-11-28T10:38:49.206Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Identify the different services available for ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13934,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/ffef5eb3-a15e...,skill/competence,sector-specific,remediate healthcare user's occupational perfo...,restore healthcare user's occupational perform...,NaN,released,2024-02-09T17:29:15.235Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,"Remediate or restore the cognitive, sensorimot..."
13935,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff0b074-5a76...,skill/competence,sector-specific,install transport equipment lighting,install transport equipment illumination\nfix ...,NaN,released,2024-02-09T17:29:14.494Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Install lighting elements in transport equipme...
13936,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff0e2cd-d0bd...,knowledge,sector-specific,natural language processing,NLP,NaN,released,2024-02-09T17:29:15.235Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,The technologies which enable ICT devices to u...
13937,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff5bc45-b506...,skill/competence,cross-sector,coordinate construction activities,reviewing construction progress\nconstruction ...,NaN,released,2024-02-09T17:29:13.7Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Coordinate the activities of several construct...


In [4]:
df_esco_jobs = pd.read_csv('data/occupations_en.csv')
df_esco_jobs

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,code
0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2654,technical director,technical and operations director\nhead of tec...,NaN,released,2024-01-25T11:28:50.295Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Technical directors realise the artistic visio...,2654.1.7
1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,metal drawing machine operator,metal drawing machine technician\nmetal drawin...,NaN,released,2024-01-23T10:09:32.099Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Metal drawing machine operators set up and ope...,8121.4
2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,precision device inspector,inspector of precision instruments\nprecision ...,NaN,released,2024-01-25T15:00:12.188Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Precision device inspectors make sure precisio...,7543.10.3
3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,air traffic safety technician,air traffic safety electronics hardware specia...,NaN,released,2024-01-29T16:01:13.998Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Air traffic safety technicians provide technic...,3155.1
4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,hospitality revenue manager,hospitality revenues manager\nyield manager\nh...,NaN,released,2024-01-11T10:28:45.871Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Hospitality revenue managers maximise revenue ...,2431.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3034,Occupation,http://data.europa.eu/esco/occupation/ff656b3a...,2120,demographer,demography research analyst\ndemography studie...,NaN,released,2024-01-23T10:55:28.923Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/8ae4...,Demographers study a variety of parameters rel...,2120.2
3035,Occupation,http://data.europa.eu/esco/occupation/ff8d4065...,9612,sorter labourer,sorter laborer\ngrader\nyard labourer\nrecycle...,NaN,released,2024-01-16T11:18:17.704Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Sorter labourers sort recyclable materials and...,9612.2
3036,Occupation,http://data.europa.eu/esco/occupation/ffa4dd5d...,5414,armoured car guard,armoured truck escort\ntruck escort\narmored c...,NaN,released,2022-01-13T09:51:50.643Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Armoured car guards ensure the safe transporta...,5414.1.2
3037,Occupation,http://data.europa.eu/esco/occupation/ffade2f4...,2422,civil service administrative officer,government administrative officer\nlocal autho...,NaN,released,2024-01-16T14:38:39.909Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Civil service administrative officers perform ...,2422.1


In [5]:
job_link_desc = {}
for row in df_esco_jobs.itertuples():
    job_link_desc[row.conceptUri] = row.description

In [6]:
skill_link_desc = {}
for row in df_esco_skills.itertuples():
    skill_link_desc[row.conceptUri] = row.description

In [7]:
# !pip install -U transformers > /dev/null

In [8]:
df = pd.read_csv('taskB/training/job2skill.tsv', sep='\t', names=['job', 'skill', 'category'])
df

,job,skill,category
0,http://data.europa.eu/esco/occupation/00030d09...,http://data.europa.eu/esco/skill/93a68dcb-3dc6...,essential
1,http://data.europa.eu/esco/occupation/00030d09...,http://data.europa.eu/esco/skill/05bc7677-5a64...,essential
2,http://data.europa.eu/esco/occupation/00030d09...,http://data.europa.eu/esco/skill/860be36a-d19b...,essential
3,http://data.europa.eu/esco/occupation/00030d09...,http://data.europa.eu/esco/skill/fed5b267-73fa...,essential
4,http://data.europa.eu/esco/occupation/00030d09...,http://data.europa.eu/esco/skill/f64fe2c2-d090...,essential
...,...,...,...
114694,http://data.europa.eu/esco/occupation/ffc292f1...,http://data.europa.eu/esco/skill/1a3660c2-011c...,optional
114695,http://data.europa.eu/esco/occupation/ffc292f1...,http://data.europa.eu/esco/skill/5b264a14-d4db...,optional
114696,http://data.europa.eu/esco/occupation/ffc292f1...,http://data.europa.eu/esco/skill/ed938748-60ae...,optional
114697,http://data.europa.eu/esco/occupation/ffc292f1...,http://data.europa.eu/esco/skill/92f54ff7-f6d8...,optional


In [9]:
queries = "taskB/validation/queries"
corpus_elements = "taskB/validation/corpus_elements"
queries = pd.read_csv(queries,sep="\t")
corpus_elements = pd.read_csv(corpus_elements, sep="\t")

corpus_elements["skill_aliases"] = corpus_elements["skill_aliases"].apply(lambda x: ast.literal_eval(x))

queries_ids = queries.q_id.to_list()
queries_texts = queries.jobtitle.to_list()
val_jobs = queries.jobtitle.to_list()
val_skills = [s[0] for s in corpus_elements.skill_aliases.to_list()]
map_queries = dict(zip(queries_ids,queries_texts))
map_queries_rev = dict(zip(queries_texts,queries_ids))

list_aliases_df = corpus_elements.explode("skill_aliases")


corpus_ids = list_aliases_df.c_id.to_list()
corpus_texts = list_aliases_df.skill_aliases.to_list()
map_corpus = dict(zip(corpus_texts,corpus_ids))
val_skill_to_link = dict(zip(corpus_texts, list_aliases_df.esco_uri.to_list()))

In [10]:
with open('taskB/training/jobid2terms.json', 'r') as f:
    jobid2name = json.load(f)

with open('taskB/training/skillid2terms.json', 'r') as f:
    skillid2name = json.load(f)

with open('/kaggle/input/train_descriptions_individual.json', 'r') as f:
    train_descs = json.load(f)

with open('/kaggle/input/val_descriptions2.json', 'r') as f:
    val_descs = json.load(f)

In [11]:
all_skills = []
all_skills_dict = {}

for link, aliases in skillid2name.items():
    all_skills.extend(aliases)
    for alias in aliases:
        all_skills_dict[alias] = link

len(all_skills)

100477

In [12]:
all_jobs = []
all_jobs_dict = {}

for link, aliases in jobid2name.items():
    all_jobs.extend(aliases)
    for alias in aliases:
        all_jobs_dict[alias] = link

len(all_jobs)

33412

In [13]:
jobs = [jobid2name[l][0] for l in df.job.unique()]
skills = [skillid2name[l][0] for l in df.skill.unique()]

In [14]:
multindata_dict = {}

for row in df.itertuples():
    job_data = multindata_dict.get(row.job, [])
    job_data.append(row.skill)
    multindata_dict[row.job] = job_data

multindata = [(k, v) for k,v in multindata_dict.items()]

In [15]:
df_jobs = pd.read_csv('taskB/validation/queries', sep='\t')
df_jobs

,q_id,jobtitle
0,dev_qb_jt_1,corporate governance analyst
1,dev_qb_jt_2,logistics business analyst
2,dev_qb_jt_3,operations planning analyst
3,dev_qb_jt_4,real estate data analyst
4,dev_qb_jt_5,security analyst ii
...,...,...
299,dev_qb_jt_300,hr generalist consultant
300,dev_qb_jt_301,data analyst apprenticeship
301,dev_qb_jt_302,data science graduate intern
302,dev_qb_jt_303,quality specialist iii


In [16]:
df_skills = pd.read_csv('taskB/validation/corpus_elements', sep='\t')
df_skills

,c_id,esco_uri,skill_aliases
0,dev_cb_sk_1,http://data.europa.eu/esco/skill/1c460d2d-90c6...,"['pricing plans', 'price strategies', 'pricing..."
1,dev_cb_sk_2,http://data.europa.eu/esco/skill/301a6581-e983...,"['putting out fires', 'put out fires', 'exting..."
2,dev_cb_sk_3,http://data.europa.eu/esco/skill/a4881e54-6055...,"['online assessment', 'analysis of web strateg..."
3,dev_cb_sk_4,http://data.europa.eu/esco/skill/efda73b4-5212...,"['keeping up with trends', 'keep pace with tre..."
4,dev_cb_sk_5,http://data.europa.eu/esco/skill/22a173f5-868c...,"['prepare tax returns form', 'complete tax ret..."
...,...,...,...
1434,dev_cb_sk_1435,http://data.europa.eu/esco/skill/113b4428-0a31...,"['keep promises', 'show reliability', 'demonst..."
1435,dev_cb_sk_1436,http://data.europa.eu/esco/skill/9d9ba780-4aa2...,"['present new employees', 'introducing new emp..."
1436,dev_cb_sk_1437,http://data.europa.eu/esco/skill/a8d41cca-3bc9...,"['prepare design specifications', 'design spec..."
1437,dev_cb_sk_1438,http://data.europa.eu/esco/skill/5100aeb3-948d...,"['define evaluation needs', 'define evaluation..."


In [17]:
df_labels = pd.read_csv('taskB/validation/qrels.tsv', sep='\t', names=['q_id', 'reserved', 'c_id', 'relevant'])
df_labels

,q_id,reserved,c_id,relevant
0,dev_qb_jt_263,0,dev_cb_sk_1009,1
1,dev_qb_jt_263,0,dev_cb_sk_1014,1
2,dev_qb_jt_263,0,dev_cb_sk_1027,1
3,dev_qb_jt_263,0,dev_cb_sk_1028,1
4,dev_qb_jt_263,0,dev_cb_sk_1069,1
...,...,...,...,...
25883,dev_qb_jt_292,0,dev_cb_sk_910,1
25884,dev_qb_jt_292,0,dev_cb_sk_930,1
25885,dev_qb_jt_292,0,dev_cb_sk_976,1
25886,dev_qb_jt_292,0,dev_cb_sk_99,1


In [18]:
# jobbert = SentenceTransformer("TechWolf/JobBERT-v2").to(device)

In [19]:
if encoder_type=='esco':
    tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large")
    encoder = AutoModel.from_pretrained("jjzha/esco-xlm-roberta-large").to(device)
elif encoder_type=='e5':
    tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-v2')
    encoder = AutoModel.from_pretrained('intfloat/e5-large-v2').to(device)
elif encoder_type=='instruct':
    tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large-instruct')
    encoder = AutoModel.from_pretrained('intfloat/multilingual-e5-large-instruct', device_map='auto')

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [20]:
qrels_file = "taskB/validation/qrels.tsv"
run_file = "validation_sentence_taskB.trec"

def get_mAP(similarities, name='Team_B_exp'):
    results = []
    results_name = []
    
    for q_idx, q_id in enumerate(queries_ids):
        sorted_indices = np.argsort(-similarities[q_idx])
        used_doc_ids = set()
        rank_counter = 0
        for c_idx in sorted_indices:  # Consider the full list.
            doc_id = corpus_ids[c_idx]
            # If doc_id was already processed, go to the next one.
            if doc_id in used_doc_ids:
                continue
            used_doc_ids.add(doc_id)
            rank_counter += 1
    
            query_name = map_queries[q_id]
            doc_name = corpus_texts[c_idx]
            score = similarities[q_idx, c_idx]
    
            results.append(f"{q_id} Q0 {doc_id} {rank_counter} {score:.4f} {name}")
            results_name.append(f"{query_name} Q0 {doc_name} {rank_counter} {score:.4f} {name}")
    
    with open("validation_sentence_taskB.trec", "w", encoding="utf-8") as f:
        f.write("\n".join(results))

    command = ["python", "talentclef25_evaluation_script/talentclef_evaluate.py", "--qrels", qrels_file, "--run", run_file]
    result = subprocess.run(command, capture_output=True, text=True)

    return float(result.stdout.split('\n')[8].split(':')[1])

In [21]:
job_template = 'it is an example of a skill related to the job: {job}'
skill_template = 'skill: {skill}'

task = 'Given a job title, retreive skills that may be relevant to the job and its associated responsibilities'

def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def get_embeddings(texts, encoder, tokenizer, batch_size=256, max_length=64, token='mean'):
    text_embeddings = torch.tensor([], device=device)
    for i in tqdm(range(0, len(texts), batch_size)):
        tokenized = {k:v.to(device) for k,v in tokenizer(texts[i:i+batch_size], max_length=max_length, padding=True, truncation=True, return_tensors='pt').items()}
        with torch.no_grad():
            outputs = encoder(**tokenized)
        if token=='mean':
            embeddings = average_pool(outputs.last_hidden_state, tokenized['attention_mask'])
        else:
            embeddings = outputs.last_hidden_state[:,0,:]
        text_embeddings = torch.cat((text_embeddings, embeddings), dim=0)
    return text_embeddings

def get_detailed_instruct(query):
    return f'Instruct: {task}\nQuery: {query}'

In [22]:
# replace jobbert with esco-xlm-roberta-large
def esco_encode_batch(texts):
    features = esco_xlm_token(texts, padding=True, max_length=512, truncation=True, return_tensors='pt')
    features = {k:v.to(device) for k,v in features.items()}
    with torch.no_grad():
        out_features = esco_xlm(**features)
    return average_pool(out_features.last_hidden_state, features['attention_mask']).cpu().numpy()

def esco_encode(texts, batch_size = 1024):
    # Sort texts by length and keep track of original indices
    sorted_indices = np.argsort([len(text) for text in texts])
    sorted_texts = [texts[i] for i in sorted_indices]
    
    embeddings = []
    
    # Encode in batches
    for i in tqdm(range(0, len(sorted_texts), batch_size)):
        batch = sorted_texts[i:i+batch_size]
        embeddings.append(esco_encode_batch(batch))
    
    # Concatenate embeddings and reorder to original indices
    sorted_embeddings = np.concatenate(embeddings)
    original_order = np.argsort(sorted_indices)
    return sorted_embeddings[original_order]

In [23]:
def encode_batch(texts):
    features = jobbert.tokenize(texts)
    features = batch_to_device(features, jobbert.device)
    features["text_keys"] = ["anchor"]
    with torch.no_grad():
        out_features = jobbert.forward(features)
    return out_features["sentence_embedding"].cpu().numpy()

def encode(texts, batch_size = 1024):
    # Sort texts by length and keep track of original indices
    sorted_indices = np.argsort([len(text) for text in texts])
    sorted_texts = [texts[i] for i in sorted_indices]
    
    embeddings = []
    
    # Encode in batches
    for i in tqdm(range(0, len(sorted_texts), batch_size)):
        batch = sorted_texts[i:i+batch_size]
        embeddings.append(encode_batch(batch))
    
    # Concatenate embeddings and reorder to original indices
    sorted_embeddings = np.concatenate(embeddings)
    original_order = np.argsort(sorted_indices)
    return sorted_embeddings[original_order]

In [24]:
# all_job_embedding = encode(all_jobs)
# # all_skill_embedding = encode(all_skills)
# query_embedding = encode(queries_texts)
# # corpus_embedding = encode(corpus_texts)

# job_sim = cos_sim(query_embedding, all_job_embedding)
# # skill_sim = cos_sim(corpus_embedding, all_skill_embedding)

In [25]:
def get_matches_job(job_index, n=3, cutoff=0.7):
    # embeddings = encode([job])
    # sim = cos_sim(embeddings, all_job_embedding).squeeze()
    sim = job_sim[job_index].squeeze()
    matches = np.argsort(-sim)[:n]
    return [all_jobs[i] for i in matches if sim[i]>cutoff]

def get_matches_skill(skill_index, n=3, cutoff=0.7):
    # embeddings = encode([skill])
    # sim = cos_sim(embeddings, all_skill_embedding).squeeze()
    sim = skill_sim[skill_index].squeeze()
    matches = np.argsort(-sim)[:n]
    return [all_skills[i] for i in matches if sim[i]>cutoff]

In [26]:
# job_template_desc = 'query: find skills related to the job {job} and its similar jobs and descriptions.'
# skill_template_desc = 'passage: The skill {skill} has the following similar skills and their descriptions.'
task = 'Given a job title and its description, retrieve relevant skills based on their descriptions'
# task = 'Given a job title, retrieve relevant skills'


if encoder_type == 'esco':
    job_template = 'Job: {job}'
    skill_template = 'Skill: {skill}'
elif encoder_type == 'e5':
    job_template = 'query: Job: {job}'
    skill_template = 'passage: Skill: {skill}'
elif encoder_type=='instruct':
    job_template = f'Instruct: {task}\nQuery: ' + 'Job: {job}'
    skill_template = 'Skill: {skill}'
# job_template_desc = 'query: Job: {job} [SEP] Description: {desc}'
# skill_template_desc = 'passage: Skill: {skill} [SEP] Description: {desc}'

if encoder_type == 'esco':
    job_template_desc = 'Job: {job} [SEP] Description: {desc}'
    skill_template_desc = 'Skill: {skill} [SEP] Description: {desc}'
elif encoder_type == 'e5':
    job_template_desc = 'query: Job: {job} [SEP] Description: {desc}'
    skill_template_desc = 'passage: Skill: {skill} [SEP] Description: {desc}'
elif encoder_type=='instruct':
    job_template_desc = f'Instruct: {task}\nQuery: ' + 'Job: {job} [SEP] Description: {desc}'
    skill_template_desc = 'Skill: {skill} [SEP] Description: {desc}'

# job_template_desc = '{job}'
# skill_template_desc = '{skill}'

# task = 'Given a job title and its related description, retreive skill descriptions that may be relevant to the job and its associated responsibilities'

data = {'job': {'name_to_link': all_jobs_dict,
                 'matcher': get_matches_job,
                 'all_items': all_jobs,
                 'template': job_template,
                 'link_to_desc':job_link_desc},
        'skill': {'name_to_link': all_skills_dict,
                  'matcher': get_matches_skill,
                  'all_items': all_skills,
                  'template': skill_template_desc,
                  'link_to_desc':skill_link_desc}}


def get_descs(item, item_type, matcher='jobbert', n=1, cutoff=0.7, index=None, desc=True):
    if matcher=='jobbert':
        if desc:
            matches = set([data[item_type]['name_to_link'][match] for match in data[item_type]['matcher'](index, cutoff=cutoff, n=n)])
        else:
            matches = set(data[item_type]['matcher'](index, cutoff=cutoff, n=n))
    else:
        matches = set([data[item_type]['name_to_link'][match] for match in difflib.get_close_matches(item, data[item_type]['all_items'], n=n, cutoff=cutoff)])
    current_prompt = data[item_type]['template'].format(**{item_type: item})
    for match in matches:
        if desc:
            # current_prompt = current_prompt + ' [SEP] Description: ' + data[item_type]['link_to_desc'][match]
            current_prompt = current_prompt + ' [SEP] Description: ' + data[item_type]['link_to_desc'][match]
        else:
            current_prompt = current_prompt + ' [SEP] ' + match
    return current_prompt

def job_get_desc(job):
    return job_template_desc.format(job=job, desc=val_descs[job])

def skill_get_desc(skill):
    return skill_template_desc.format(skill=skill, desc=skill_link_desc[val_skill_to_link[skill]])

def get_detailed_instruct_desc(query):
    return f'Instruct: {task}\nQuery: {job_get_desc(query)}'

In [27]:
def get_validation_mAP(encoder, tokenizer, desc=True, **kwargs):
    # if kwargs['n']==0:
    #     jobs_with_desc = queries_texts
    #     # skills_with_desc = corpus_texts
    # else:
    
    # jobs_with_desc = [get_descs(j, 'job', index=i, **kwargs) for i, j in enumerate(tqdm(queries_texts))]
    if desc:
        jobs_with_desc = [job_get_desc(j) for j in queries_texts]
        skills_with_desc = [skill_get_desc(s) for s in corpus_texts]
    else:
        jobs_with_desc = [job_template.format(job=j) for j in queries_texts]
        skills_with_desc = [skill_template.format(skill=s) for s in corpus_texts]
        # jobs_with_desc = queries_texts
        # skills_with_desc = corpus_texts
    job_embeddings = get_embeddings(jobs_with_desc, encoder=encoder, tokenizer=tokenizer, max_length=128, token='mean', batch_size=64)
    skill_embeddings = get_embeddings(skills_with_desc, encoder=encoder, tokenizer=tokenizer, max_length=128, token='mean', batch_size=64)
    similarities = cos_sim(job_embeddings, skill_embeddings).cpu().numpy()
    return get_mAP(similarities)

In [28]:
class MultiNDataset(torch.utils.data.Dataset):
    def __init__(self, pairs, encoder='e5', ):
        self.pairs = pairs
        self.encoder = encoder

    def __getitem__(self, idx):
        job_id, skill_ids = self.pairs[idx]
        job_alias = np.random.choice(jobid2name[job_id])
        skill_id = np.random.choice(skill_ids)
        # job = np.random.choice(jobid2name[job_id])
        # skill = np.random.choice(skillid2name[skill_id])
        # job = 'query: Job: ' + np.random.choice(jobid2name[job_id]) + ' [SEP] Description: ' + train_descs[job_id]
        # skill = 'passage: Skill:' + np.random.choice(skillid2name[skill_id]) + ' [SEP] Description: ' + skill_link_desc[skill_id]
        # job = 'Instruct: Given a job title, retrieve relevant skills based on their descriptions\nQuery: Job: ' + np.random.choice(jobid2name[job_id])
        if self.encoder == 'esco':
            job = 'Job: ' + job_alias + ' [SEP] Description: ' + train_descs[job_alias]
            skill = 'Skill:' + np.random.choice(skillid2name[skill_id]) + ' [SEP] Description: ' + skill_link_desc[skill_id]
        elif self.encoder == 'e5':
            job = 'query: Job: ' + job_alias + ' [SEP] Description: ' + train_descs[job_alias]
            skill = 'passage: Skill:' + np.random.choice(skillid2name[skill_id]) + ' [SEP] Description: ' + skill_link_desc[skill_id]
        elif encoder_type=='instruct':
            job = 'Instruct: Given a job title and its description, retrieve relevant skills based on their descriptions\nQuery: Job: ' + job_alias + ' [SEP] Description: ' + train_descs[job_alias]
            skill = 'Skill:' + np.random.choice(skillid2name[skill_id]) + ' [SEP] Description: ' + skill_link_desc[skill_id]
        # if self.encoder == 'esco':
        #     job = 'Job: ' + job_alias 
        #     skill = 'Skill:' + np.random.choice(skillid2name[skill_id])
        # elif self.encoder == 'e5':
        #     job = 'query: Job: ' + job_alias
        #     skill = 'passage: Skill:' + np.random.choice(skillid2name[skill_id])
        # elif self.encoder == 'instruct':
        #     job = 'Instruct: Given a job title, retrieve relevant skills\nQuery: Job: ' + job_alias 
        #     skill = 'Skill:' + np.random.choice(skillid2name[skill_id])

        # if self.encoder == 'esco':
        #     job = 'Job: ' + job_alias + ' [SEP] Description: ' + job_link_desc[job_id]
        #     skill = 'Skill:' + np.random.choice(skillid2name[skill_id]) + ' [SEP] Description: ' + skill_link_desc[skill_id]
        # elif self.encoder == 'e5':
        #     job = 'query: Job: ' + job_alias + ' [SEP] Description: ' + job_link_desc[job_id]
        #     skill = 'passage: Skill:' + np.random.choice(skillid2name[skill_id]) + ' [SEP] Description: ' + skill_link_desc[skill_id]
        # elif self.encoder == 'instruct':
        #     job = 'Instruct: Given a job title and its description, retrieve relevant skills based on their descriptions\nQuery: Job: ' + job_alias + ' [SEP] Description: ' + job_link_desc[job_id]
        #     skill = 'Skill:' + np.random.choice(skillid2name[skill_id]) + ' [SEP] Description: ' + skill_link_desc[skill_id]
        
        item = {'job': job, 'skill': skill}
        return item

    def __len__(self):
        return len(self.pairs)

def custom_collate_fn(batch, tokenizer, max_length):
    """
    Takes a list of dictionaries from JobSkillDataset.__getitem__.
    Tokenizes job and skill texts separately and pads them dynamically per batch.
    """
    job_texts = [item['job'] for item in batch]
    skill_texts = [item['skill'] for item in batch]

    # Tokenize jobs and skills separately
    tokenized_jobs = tokenizer(job_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    tokenized_skills = tokenizer(skill_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

    return {
        'jobs': tokenized_jobs,
        'skills': tokenized_skills
    }

train_multin_dataset = MultiNDataset(multindata, encoder=encoder_type)

In [29]:
collate_wrapper = partial(custom_collate_fn, tokenizer=tokenizer, max_length=256)

In [30]:
batch_size = 4

train_loader = DataLoader(train_multin_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_wrapper)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [31]:
best_val_mAP = 0

In [32]:
# saved_data = torch.load('best_model_e5_large_llm_desc_mAP.pt')
# # encoder.load_state_dict(saved_data['weights'])
# best_val_mAP = saved_data["mAP"]
# print(f'Saved val mAP: {saved_data["mAP"]}')

In [33]:
num_epochs = 20
gradient_accumulation_steps = 16

# optimizer_grouped_parameters = [
#     {
#         "params": model.encoder.parameters(),
#         "lr": 1e-5,
#     },
#     {
#         "params": model.classifier.parameters(),
#         "lr": 1e-3,
#     },
# ]
optim = torch.optim.AdamW(encoder.parameters(), lr=1e-5, betas=(0.9, 0.98))
# optim = torch.optim.AdamW(optimizer_grouped_parameters, betas=(0.9, 0.98))
num_training_steps = (len(train_loader) * num_epochs) // gradient_accumulation_steps
num_warmup_steps = int(0.06 * num_training_steps)  # 6% warmup

scheduler = get_linear_schedule_with_warmup(
    optim,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)

In [34]:
def calculate_mnrl_loss(job_embeds, skill_embeds, temperature):
     # Normalize embeddings for stable cosine similarity calculation
    job_embeds = F.normalize(job_embeds, p=2, dim=1)
    skill_embeds = F.normalize(skill_embeds, p=2, dim=1)

    # Calculate cosine similarity matrix (num_jobs x num_skills)
    # Assumes job_embeds[i] corresponds to skill_embeds[i] as the positive pair
    similarity_matrix = torch.matmul(job_embeds, skill_embeds.T) # (batch_size, batch_size)

    # Scale similarities by temperature
    similarity_matrix = similarity_matrix / temperature

    # Create labels: Target for job_i is skill_i (the diagonal element)
    batch_size = job_embeds.size(0)
    labels = torch.arange(batch_size, dtype=torch.long, device=job_embeds.device)

    # Calculate CrossEntropyLoss
    # The loss encourages the diagonal similarity (positive pair) to be higher
    # than the off-diagonal similarities (negative pairs in the batch)
    loss = F.cross_entropy(similarity_matrix, labels)
    return loss

In [35]:
# get_validation_mAP(encoder, tokenizer, desc=False)

In [36]:
global_step = 0

encoder.zero_grad()
for epoch in range(num_epochs):
    encoder.train()
    train_loss = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True) # Keep pbar after loop
    for step, batch in enumerate(pbar):
        # Move batch to device (collate_fn already returns tensors)
        job_inputs = {k: v.to(device) for k, v in batch['jobs'].items()}
        skill_inputs = {k: v.to(device) for k, v in batch['skills'].items()}

        # Forward pass through the base model
        job_outputs = encoder(**job_inputs)
        skill_outputs = encoder(**skill_inputs)

        # Get embeddings using mean pooling
        job_embeddings = average_pool(job_outputs.last_hidden_state, job_inputs['attention_mask'])
        skill_embeddings = average_pool(skill_outputs.last_hidden_state, skill_inputs['attention_mask'])

        # Calculate custom MNRL loss
        loss = calculate_mnrl_loss(job_embeddings, skill_embeddings, temperature=0.05)

        # --- Gradient Accumulation ---
        # 1. Scale loss
        loss = loss / gradient_accumulation_steps

        # 2. Backward pass
        loss.backward()

        train_loss += loss.item() * gradient_accumulation_steps # Unscale for logging

        # 3. Optimizer step (conditional)
        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=1.0) # Optional gradient clipping
            optim.step()
            scheduler.step()
            optim.zero_grad() # Reset gradients *after* optimizer step
            global_step += 1

            # Update progress bar description with averaged loss for the epoch so far
            avg_loss_so_far = train_loss / (step + 1)
            pbar.set_postfix({'loss': f"{avg_loss_so_far:.4f}", 'lr': f"{scheduler.get_last_lr()[0]:.2e}"})


    # --- End of Epoch ---
    avg_epoch_loss = train_loss / len(train_loader)
    
    encoder.eval()
    # validation_mAP = get_validation_mAP(encoder, tokenizer, n=1, desc=True, cutoff=0.9)
    validation_mAP = get_validation_mAP(encoder, tokenizer, desc=True)
    
    print(f'Epoch {epoch+1} finished. Train loss: {avg_epoch_loss}, Validation mAP: {validation_mAP}')
    if validation_mAP>best_val_mAP:
        torch.save({'mAP': validation_mAP, 'weights': encoder.state_dict()}, f'best_model_{encoder_type}_l_llm_indiv_desc_mAP.pt')
        best_val_mAP = validation_mAP

100%|██████████| 156/156 [01:12<00:00,  2.15it/s]


Epoch 1 finished. Train loss: 0.6796127066404557, Validation mAP: 0.2879


100%|██████████| 156/156 [01:12<00:00,  2.15it/s]


Epoch 2 finished. Train loss: 0.4861311547156389, Validation mAP: 0.2965


100%|██████████| 156/156 [01:12<00:00,  2.15it/s]


...

Epoch 18 finished. Train loss: 0.3485908088260231, Validation mAP: 0.2908


100%|██████████| 156/156 [01:12<00:00,  2.15it/s]


Epoch 19 finished. Train loss: 0.3289681336615444, Validation mAP: 0.2922


100%|██████████| 156/156 [01:12<00:00,  2.15it/s]


Epoch 20 finished. Train loss: 0.34101611860486053, Validation mAP: 0.2932


In [37]:
best_val_mAP

0.3089

In [38]:
# get_validation_mAP(encoder, tokenizer)
# get_validation_mAP(model.encoder, model.tokenizer)